#### Stimulus decoding

In [1]:
import sys

sys.path.insert(1, "/home/vinicius/storage1/projects/GrayData-Analysis")

In [2]:
import itertools
import os

import matplotlib
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xarray as xr
from frites.conn.conn_spec import conn_spec
from frites.conn.conn_tf import _tf_decomp
from frites.utils import parallel_func
from scipy.signal import find_peaks
from tqdm import tqdm

from config import sessions
from GDa.session import session, session_info
from GDa.signal.surrogates import trial_swap_surrogates
from GDa.temporal_network import temporal_network
from GDa.util import _extract_roi, average_stages, create_stages_time_grid

In [3]:
import keras
from keras import metrics
from keras.layers import Conv2D, Dense, Dropout, Flatten, Input, MaxPool2D
from keras.models import Model, Sequential
from keras.utils import np_utils

2022-07-05 20:36:21.073664: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-05 20:36:21.095188: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/vinicius/tools/llvm-project/build/lib:
2022-07-05 20:36:21.095230: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [8]:
tt = br = avg = 1
at = "cue"
_ROOT = os.path.expanduser("~/funcog/gda")

In [14]:
sxx = []
stim = []
i = 0
for s_id in tqdm(sessions):
    _FILE_NAME = f"power_tt_{tt}_br_{br}_at_{at}.nc"
    path_pow = os.path.join(_ROOT, f"Results/lucy/{s_id}/session01", _FILE_NAME)
    power = xr.load_dataarray(path_pow)
    # Averages power for each period (baseline, cue, delay, match) if needed
    out = average_stages(power, avg)

    sxx += [out.sel(roi="V1")]
    stim += [out.attrs["stim"].astype(int) + 5 * i] * len(out["roi"])
    i = i + 1

100%|████████████████████████████████████████████████████████| 62/62 [07:13<00:00,  6.99s/it]


In [32]:
x = xr.concat(sxx, "roi")

In [36]:
x_train
for i in range(5):
    x.isel(freqs=5, times=i).values
    x[~np.isnan(x)].shape

(343289,)

In [4]:
def load_session_data(sid):

    # Instantiate class
    ses = session(
        raw_path=os.path.expanduser("~/funcog/gda/GrayLab/"),
        monkey="lucy",
        date=sid,
        session=1,
        slvr_msmod=False,
        align_to="cue",
        evt_dt=[-0.65, 3.00],
    )

    # Read data from .mat files
    ses.read_from_mat()

    # Filtering by trials
    data = ses.filter_trials(trial_type=[1], behavioral_response=[1])
    # ROIs with channels
    rois = [
        f"{roi} ({channel})"
        for roi, channel in zip(data.roi.data, data.channels_labels)
    ]
    data = data.assign_coords({"roi": rois})

    return data

In [5]:
sid = "141024"

data = [load_session_data(sid) for sid in [sid]][0]

In [6]:
# Defining parameters
decim = 20  # Downsampling factor
mode = "multitaper"  # Wheter to use Morlet or Multitaper
n_freqs = 80  # How many frequencies to use
fc = np.linspace(3, 75, n_freqs)  # Frequency array
n_cycles = fc / 4  # Number of cycles
mt_bandwidth = None

bands = {
    "theta": [0, 6],
    "alpha": [6, 14],
    "beta_1": [14, 26],
    "beta_2": [26, 43],
    "gamma": [43, 80],
}

w = _tf_decomp(
    data,
    data.attrs["fsample"],
    fc,
    mode=mode,
    n_cycles=n_cycles,
    mt_bandwidth=None,
    decim=decim,
    kw_cwt={},
    kw_mt={},
    n_jobs=20,
)

w = xr.DataArray(
    w,
    name="power",
    dims=("trials", "roi", "freqs", "times"),
    coords=(data.trials.values, data.roi.values, fc, data.time.values[::decim]),
)

# Power
power = (w * np.conj(w)).real

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  52 tasks      | elapsed:   53.0s
[Parallel(n_jobs=20)]: Done  75 out of  78 | elapsed:  1.2min remaining:    2.8s
[Parallel(n_jobs=20)]: Done  78 out of  78 | elapsed:  1.2min finished


NameError: name 'beta' is not defined

In [7]:
power_beta = power.sel(freqs=slice(24, 43)).mean("freqs")

#### Creating classifier

In [19]:
X = power_beta.data.swapaxes(1, 2).reshape(582 * 183, 78)

In [35]:
y = np.tile(data.stim, 183)

In [36]:
x_train, y_train = X[:50000], y[:50000]
x_test, y_test = X[50000:], y[50000:]

In [37]:
y_train = np_utils.to_categorical(y_train - 1, num_classes=5)
y_test = np_utils.to_categorical(y_test - 1, 5)

In [38]:
input_shape = (80, 183, 1)

In [41]:
model = Sequential()

model.add(Dense(5000, input_dim=78, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1000, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(500, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(50, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(5, activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["acc"])

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 5000)              395000    
                                                                 
 dropout_22 (Dropout)        (None, 5000)              0         
                                                                 
 dense_28 (Dense)            (None, 1000)              5001000   
                                                                 
 dropout_23 (Dropout)        (None, 1000)              0         
                                                                 
 dense_29 (Dense)            (None, 500)               500500    
                                                                 
 dropout_24 (Dropout)        (None, 500)               0         
                                                                 
 dense_30 (Dense)            (None, 50)               

In [42]:
h = model.fit(x_train, y_train, epochs=10, batch_size=128, verbose=1)

Epoch 1/10
275/391 [====================>.........] - ETA: 3s - loss: 1.5986 - acc: 0.2371

KeyboardInterrupt: 

In [34]:
model.predict(x_test).argmax(-1).max()

1766/1766 [==============================] - 6s 4ms/step


1